## Overview ##
---
Download Basemap quads based on the mosaic name, merged and clipped to area of interest.

For example, from Explorer, I've defined my area of interest and downloaded that geoJSON.

I've selected the basemap:

global_monthly_2022_01_mosaic

and downloaded my Area of Interest.

To view the basemap, I clicked on the arrow-in-a-box icon to open that basemap in the Basemap viewer.

To see all quads in my subject area, I uploaded my Area of Interest.

326-1257,327-1257,328-1257,326-1256,327-1256,328-1256,326-1255,327-1255,328-1255


<a name="Get-a-basemap-ID"></a> 
## Get a basemap ID
---

You can get a basemap ID from Planet Explorer or the Basemap Viewer.

### Get a basemap ID from Planet Explorer
You&#8217;ll need a basemap ID to work identify what basemap to download.

To pick a basemap, you can open <a href="https://www.planet.com/explorer/#" target="_blank">Planet Explorer</a> and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for basemaps or speak to your customer service manager.)

Once you've selected a basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example monthly or quarterly. Then select one of the resulting basemap IDs, such as "global_monthly_2022_05_mosaic." That's the value you'll be passing into the Orders API. 

### Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the Basemap Viewer and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want.

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as "global_monthly_2022_01_mosaic." That's the value you'll be passing into the Orders API.

<a name="how-to-get-started"></a> 
## Start coding

---
Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

In [17]:
import planet
import os
from planet import Auth
from planet import OrdersClient
from planet import Session



In [18]:
API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
auth = Auth.from_key(API_KEY)


### Create an order packet
---
Package up the details of your order in a JSON object and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. 

In [19]:
order_params = {
   "name": "basemap order with geometry",
   "source_type": "basemaps",
   "order_type":"partial",
   "products": [
       {
           "mosaic_name": "global_monthly_2022_01_mosaic",
           "geometry":{
               "type": "Polygon",
               "coordinates":[
                   [
                       [ 4.607406, 52.353994 ],
                        [ 4.680005, 52.353994 ],
                        [ 4.680005, 52.395523 ],
                        [ 4.607406, 52.395523 ],
                        [ 4.607406, 52.353994 ]
                   ]
               ]
           }
      }
  ],
    "tools": [
      {"merge": {}},
      {"clip": {}}
      

  ],
 "delivery":{
      "google_cloud_storage":{
         "bucket":"notebook-practice",
         "credentials":GCP_CREDENTIALS,
         "path_prefix":"basemaps-to-cloud/"
      }


 }
}



### Create a session with SDK and poll for success

Here, we are creating a session with the SDK and creating the order using the order parameters. Then, we are polling for success. This may take a few minutes to complete.



In [20]:
with planet.reporting.StateBar(state='creating') as reporter:
    async with Session() as sess:
        client = OrdersClient(sess)
        # perform operations here
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        await client.wait(order['id'], callback=reporter.update_state,
                        max_attempts=0)

00:05 - order db5ed0e0-c8fe-48d6-ad15-a62243b47632 - state: failed 


### Download the order

Finally, we download the order using download_order from the SDK and print the result.

In [21]:
async def create_and_download(order_detail, directory, client):
    with planet.reporting.StateBar(state='creating') as reporter:
        # create
        order = await client.create_order(order_detail)
        reporter.update(state='created', order_id=order['id'])

        # wait for completion
        await client.wait(order['id'], callback=reporter.update_state)

    # download
    await client.download_order(order['id'], directory, progress_bar=True)

print(order)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/db5ed0e0-c8fe-48d6-ad15-a62243b47632'}, 'created_on': '2023-01-10T22:45:43.100Z', 'delivery': {'google_cloud_storage': {'bucket': 'notebook-practice', 'credentials': '<REDACTED>', 'path_prefix': 'basemaps-to-cloud/'}}, 'error_hints': [], 'id': 'db5ed0e0-c8fe-48d6-ad15-a62243b47632', 'last_message': 'Preparing order', 'last_modified': '2023-01-10T22:45:43.100Z', 'name': 'basemap order with geometry', 'order_type': 'partial', 'products': [{'geometry': {'coordinates': [[[4.607406, 52.353994], [4.680005, 52.353994], [4.680005, 52.395523], [4.607406, 52.395523], [4.607406, 52.353994]]], 'type': 'Polygon'}, 'mosaic_name': 'global_monthly_2022_01_mosaic'}], 'source_type': 'basemaps', 'state': 'queued', 'tools': [{'merge': {}}, {'clip': {}}]}
